In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [324]:
version = 'v2'

# R  W  11111111
# R  P  11111111
# G  W  11111111
# G  P  11111111
# H  W  11111
# H  P  11111111

# category = 'R'
# category = 'G'
category = 'H'

bet_type = 'W'
# bet_type = 'P'

In [325]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

29813 races loaded!


In [326]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

7541 H races!


In [327]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
2274,5520,YONKERS,USA,C,H,2017-09-13,2,YONKERS TROT MS,2017-09-12 23:35:00,Paying,1609,"[[3], [2], [6], [5]]",8,"[{""runnerName"": ""FAIR GLIDER"", ""runnerNumber"":...",4.1,30.4,19.8,0.0,"[[3], [2], [6], [5]]","[{'runnerName': 'FAIR GLIDER', 'runnerNumber':..."
7299,17148,EUGOWRA,NSW,S,H,2017-10-02,1,TAB.COM.AU PACE MS,2017-10-02 02:08:00,Paying,2100,"[[4], [10], [9], [3]]",10,"[{""runnerName"": ""MY DOLLY DIVA"", ""runnerNumber...",12.9,17.4,77.6,381.4,"[[4], [10], [9], [3]]","[{'runnerName': 'MY DOLLY DIVA', 'runnerNumber..."
7300,17149,EUGOWRA,NSW,S,H,2017-10-02,2,FORBES LIVESTOCK AGL ENERGY HT MS,2017-10-02 02:43:00,Paying,1700,"[[3], [1], [2], [4]]",10,"[{""runnerName"": ""HALLYCOULD"", ""runnerNumber"": ...",13.3,26.0,48.0,462.7,"[[3], [1], [2], [4]]","[{'runnerName': 'HALLYCOULD', 'runnerNumber': ..."


In [328]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    data_all = data_all.append(race['runners'])
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
7,0.094290,0.045473,0.030877,0.032593,0,0,2,"{'returnWin': 13, 'returnWinOpen': 13, 'return...",True,0.1,...,0.3,0.048514,TORBJORN JANSSON,RED ROOSTER,8,N JOHANSSON,24.5,0.040816,0.5,0.034512
8,0.031639,0.015258,0.016235,0.017137,0,0,0,"{'returnWin': 31, 'returnWinOpen': 31, 'return...",True,0.1,...,0.2,0.024046,PER LINDEROTH,TIGERDROTTNINGEN,9,P LINDEROTH,42.9,0.023310,0.1,0.019710
9,0.350466,0.169019,0.168008,0.177344,0,0,3,"{'returnWin': 8.5, 'returnWinOpen': 8.5, 'retu...",True,0.1,...,0.8,0.131680,ULF OHLSSON,LOVELY FLICKA,10,S BATH,5.0,0.200000,0.8,0.169109


In [329]:
# drop scratched
data = data_all.dropna(subset=['has_odds'])
data.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
7,0.094290,0.045473,0.030877,0.032593,0,0,2,"{'returnWin': 13, 'returnWinOpen': 13, 'return...",True,0.1,...,0.3,0.048514,TORBJORN JANSSON,RED ROOSTER,8,N JOHANSSON,24.5,0.040816,0.5,0.034512
8,0.031639,0.015258,0.016235,0.017137,0,0,0,"{'returnWin': 31, 'returnWinOpen': 31, 'return...",True,0.1,...,0.2,0.024046,PER LINDEROTH,TIGERDROTTNINGEN,9,P LINDEROTH,42.9,0.023310,0.1,0.019710
9,0.350466,0.169019,0.168008,0.177344,0,0,3,"{'returnWin': 8.5, 'returnWinOpen': 8.5, 'retu...",True,0.1,...,0.8,0.131680,ULF OHLSSON,LOVELY FLICKA,10,S BATH,5.0,0.200000,0.8,0.169109


In [330]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    75458.000000
mean         0.100135
std          0.300182
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [331]:
xn = data['num_runners']
xwp = data['win_perc']
xws = data['win_scaled']
xwr = data['win_rank']
xpp = data['place_perc']
xps = data['place_scaled']
xpr = data['place_rank']
X = pd.concat([xn, xwp, xws, xwr, xpp, xps, xpr], axis=1)
X.describe()

,num_runners,win_perc,win_scaled,win_rank,place_perc,place_scaled,place_rank
count,75458.000000,75458.000000,75458.000000,75458.000000,75458.000000,75458.000000,75458.000000
mean,0.105519,0.118497,0.099936,0.545658,0.333507,0.099936,0.551094
std,0.026002,0.143305,0.120432,0.296985,0.247085,0.077276,0.298260
min,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.090909,0.026525,0.022357,0.300000,0.138889,0.040831,0.300000
50%,0.100000,0.064516,0.054267,0.555556,0.277778,0.081524,0.555556
75%,0.125000,0.153846,0.129316,0.800000,0.476190,0.141068,0.800000
max,0.333333,0.961538,0.920801,1.000000,1.000000,0.859575,1.000000


In [332]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 500
print('epochs = {}'.format(epochs))

layer_1 = 50
layer_2 = 50

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 7
epochs = 500
tag = H50x50W
model loaded


In [ ]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)
print('model saved')